# Genrative Model

We start of by loading the required packages needed in the coding

In [15]:
#importing packages
from __future__ import print_function
import os
import re
import string
import tensorflow as tf
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from pickle import dump
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# Lesson 1: Exploring data

Firstly, we have downloaded fairy tales from the webpage Gutenberg.org. After downloading data and looking at the data, which we need to clean. We observe following
- Illustration texts
- The text file contain multiple stories, these should be split into multiple files in order for the model to know there is groupings (Extension: will be reviewed in lesson 5)
- The file contains a number of punctuations (,;.* etc.), which are irrelevant for the model
- Capital letters as headers

Next we create a function, which allows us to load the data into the script. 

In [16]:
# creating function to load documents into memory
def load_doc(filename):
	file = open(filename, 'r', encoding="utf8") #loading an existing file
	text = file.read() #opening the file and assigning it to the variable text
	file.close() #close the file
	return text #output = text

We use multiple fairy tales and concatenate them into a single file to can load them easily into the script

In [17]:
#concatenate text files
path = 'Data' #Setting the path
 
files = os.listdir(path) #assign the filenames in the directory to the variable files

#with open('Data/adventures.txt', 'w', encoding="utf8") as outfile: #writing a loop adds each fairy tale to a signle file
#    for fname in files:
#        with open(fname, encoding="utf8") as infile:
#            outfile.write(infile.read())


Applying the loading function to the created document

In [18]:
# load document
in_filename = 'Data/adventures.txt'#specifying the filename of the data we wish to load
doc = load_doc(in_filename) #loading the file
print(doc[:100]) #printing the first 200 characters of the loading document

﻿THE SNOW QUEEN

A TALE IN SEVEN STORIES

FIRST STORY

WHICH DEALS WITH A MIRROR AND ITS FRAGMENTS




In this lesson we have created a single document containing the fairy tales, we need to use for the model and created a function for loading it into the script

# Cleaning data (lesson 2)

In lesson 1, we observed several potential issues in the data, which we need to remove from the data. This is done below:

In [19]:
#removing illustration descriptions
doc = re.sub(r'\[[^)]*\]', '', doc) #using re.sub function and regular expressions 
#[ - an opening bracket 
#[^()]* - zero or more characters other than those defined, that is, any characters other than [ and ]
#\] - a closing bracket


In [20]:
#removing headers
doc = re.sub(r'[A-Z]{2,}','', doc) #replacing capital letters longer than 1 with nothing. 


In [21]:
# turn a doc into clean tokens
def clean_doc(doc): #making a function
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token - https://www.geeksforgeeks.org/python-maketrans-translate-functions/
	table = str.maketrans('', '', string.punctuation) #Third argument specifies the wished deleted items
	tokens = [w.translate(table) for w in tokens] #translate applies the translation table applied on the looping through the tokens list
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()] #Replace word with word, if the word is alphabetic
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [22]:
tokens = clean_doc(doc) #using the clean function to cleaning our document
print(tokens[:200])
print('Total Tokens: %d' % len(tokens)) #printing the number of tokens
print('Unique Tokens: %d' % len(set(tokens))) #printing the number of unique tokens by grouping similair tokens

['a', 'a', 'at', 'home', 'in', 'the', 'princes', 'palace', 'when', 'at', 'night', 'the', 'others', 'were', 'asleep', 'she', 'used', 'to', 'go', 'out', 'on', 'to', 'the', 'marble', 'steps', 'it', 'cooled', 'her', 'burning', 'feet', 'to', 'stand', 'in', 'the', 'cold', 'seawater', 'and', 'at', 'such', 'times', 'she', 'used', 'to', 'think', 'of', 'those', 'she', 'had', 'left', 'in', 'the', 'deep', 'one', 'night', 'her', 'sisters', 'came', 'arm', 'in', 'arm', 'they', 'sang', 'so', 'sorrowfully', 'as', 'they', 'swam', 'on', 'the', 'water', 'that', 'she', 'beckoned', 'to', 'them', 'and', 'they', 'recognised', 'her', 'and', 'told', 'her', 'how', 'she', 'had', 'grieved', 'them', 'all', 'after', 'that', 'they', 'visited', 'her', 'every', 'night', 'and', 'one', 'night', 'she', 'saw', 'a', 'long', 'way', 'out', 'her', 'old', 'grandmother', 'who', 'for', 'many', 'years', 'had', 'not', 'been', 'above', 'the', 'water', 'and', 'the', 'merman', 'king', 'with', 'his', 'crown', 'on', 'his', 'head', 'they

We are now left with only tokens from the documents, since we have cleaned the document from headers and punctuation and split them into single words. Next we need to arrange the tokens into sequences of 50 input items and 1 output item

In [23]:
# organize into sequences of tokens
length = 50 + 1 #defining the length of the sequence
sequences = list() #creating an empty list
for i in range(length, len(tokens)): #range takes two arguments: start point and end point
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq) #-join returns a string in which the string elements of sequence have been joined by str separator
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 15523


Next we save the sequences into a file, which can be loaded for the next lesson

In [24]:
# save tokens to file, one dialog per line
def save_doc(lines, filename): #creating a saving function
	data = '\n'.join(lines) #joining the lines seperated by 'enter'
	file = open(filename, 'w', encoding="utf8") #create empty new file
	file.write(data) #input the data in the empty file
	file.close() #close the file

In [25]:
# save sequences to file
out_filename = 'Data/fairytales_sequences.txt'
save_doc(sequences, out_filename)

# Training the model (Lesson 3)

In the last lesson we obtained a document consisting of clean sequences, which is ready for use in training our model. We start by loading these sequences

In [26]:
# loading the cleaned data
in_filename = 'Data/fairytales_sequences.txt'
doc = load_doc(in_filename)

#split the text by lineshifts creating a list of 51 items long string
lines = doc.split('\n')

Next, we transform the sequences into to integers to allow us to create wordvectors

In [45]:
# integer encode sequences of words
tokenizer = Tokenizer() #assign the tokenizer function to a variable
tokenizer.fit_on_texts(lines) #function of keras finds all of the unique words in the data and assigns each a unique integer
sequences = tokenizer.texts_to_sequences(lines) #translating the input lines into integers
#tokenizer.word_index #checking the dictionary of the transformed wordsb

To estimate the size of the embedding layer in the ANN, we calculate the vocabulary size. Next we seperate the sequences into input and output, so the model will based on the 50 elements long input sequence, will be able to predict the 51th word, which is the output. 

In [28]:
#calculating vocabulary size to estimate the size of the embedding layer
vocab_size = len(tokenizer.word_index) + 1 #since indexing of array are zero-offset, the index of the vocabulary must be one larger than the length
vocab_size

2460

In [29]:
# separate into input and output
sequences = array(sequences) #transforming the sequens of integers to arrays
X, y = sequences[:,:-1], sequences[:,-1] #defining X (input sequences) and y (output words)
y = to_categorical(y, num_classes=vocab_size) #to_categorical converts a class vector (integers) to binary class matrix
seq_length = X.shape[1] #gives you the dimension of the array, which we put to be 50
seq_length

50

Next, we define the model with different layers. We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

In [30]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, 50, input_length=seq_length)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            123000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2460)              248460    
Total params: 522,360
Trainable params: 522,360
Non-trainable params: 0
_________________________________________________________________
None


After defining the model, we train it on the training sequences and save it for the next lesson alongside with the tokenizer, which contains the translation of the tokens into integers. 

In [32]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
model.fit(X, y, batch_size=128, epochs=1) #training the model, orginally 100 epoch, but changed for illustration of output 

Epoch 1/1
15523/15523 [==============================] - 44s 3ms/step - loss: 6.2251 - acc: 0.0759


In [33]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Lesson 4

In the last lesson we created a model along side with the tokenizer dictionary, which we will start of by loading into the script. We will also load the cleaned sequences, which will be used to select a input sequence to prompt the model to produce new output. 

In [20]:
# load cleaned text sequences
doc = load_doc('Data/fairytales_sequences.txt')
lines = doc.split('\n')

In [21]:
#Specifying input sequences length to prompt the model
seq_length = len(lines[3].split()) - 1 #splitting the sequences into words, counting them -1
seq_length

50

In [22]:
#loading model
model= load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb')) #r for read

In [23]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

behind them the skin of their feet the quills of their wings and the soft feathers of their breasts and when the frost had gone the salt water was torture for their wounds yet ever they sang their songs piercing sweet and speaking of the peace and joy to come and



51

After selecting a seed text to prompt the model, we will translate it into integers that the model will know how to manipulate. 

In [24]:
#translating the input text to integers
encoded = tokenizer.texts_to_sequences([seed_text])[0]
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

The model can predict the next word directly by calling model.predict_classes() that will return the index of the word with the highest probability

In [25]:
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=1)

1/1 [==============================] - 0s 353ms/step


Looking up the index in the Tokenizers mapping to get the associated word

In [26]:
#making a loop to translate integer to word
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

out_word

'measured'

The input sequences will get too long, in order to keep them to 50 items using the following function, which pads sequences to the same length

In [27]:
#Setting the max length to be 50 items by removing items from the beginnning of the sequence
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

To get the model to append the predicted word to the seed text and predict the following word by the new seed text, we create a function. 

In [43]:
#creating a function, which generates the predicted output
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list() #make an empty list
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

Now we generate an output!

In [29]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'Data/first_output.txt'
save_doc(generated, out_filename)


measured measured strings heed shrank hear hear limitless ledge penetrate penetrate penetrate bred limitless limitless limitless jammed jammed jammed redoubted redoubted sins swam redoubted swam redoubted redoubted damp redoubted redoubted dropping dropping treachery redoubted redoubted redoubted redoubted redoubted redoubted loch loch moaning moaning unexpected unexpected unexpected unexpected whelps unexpected unexpected unexpected whelps unexpected unexpected unexpected piece piece matter matter burst burst unalterable unalterable unalterable shields shields shields shields meet meet majesty majesty meet natural natural natural natural march march evade evade march fingernails march march march learned march tailleken concerning concerning concerning concerning priestly priestly hair concerning concerning concerning fly


The model appears to get stuck in one word. This is probably due to the fact, that the model has predicted the same words a couple of times in a row, which it has appended to the seed text. This is not strange considering the input sequences only change by one word. The more times it predicts the same word from the context the more likely it is it will predict it again. 

An example: Predicting 'measured' from the input sequence 'behind them the skin of their feet the quills of their wings and the soft feathers of their breasts and when the frost had gone the salt water was torture for their wounds yet ever they sang their songs piercing sweet and speaking of the peace and joy to come and' will become 'them the skin of their feet the quills of their wings and the soft feathers of their breasts and when the frost had gone the salt water was torture for their wounds yet ever they sang their songs piercing sweet and speaking of the peace and joy to come and measuring', which will lead the model to predict 'measuring' again, since the input did not differ that much. This will give the new input sequence: 'the skin of their feet the quills of their wings and the soft feathers of their breasts and when the frost had gone the salt water was torture for their wounds yet ever they sang their songs piercing sweet and speaking of the peace and joy to come and measuring measuring'. The more times measuring occurs in the end, the more likely it is it will predict 'measuring' again. 

A way to overcome this is maybe to give the model more data, make the input sequences differ more or give the model some elementary syntactic rules to progress from. 

# Lesson 5: Advancing the model


In the last lesson we were able to produce an output, though it might not have been ideal. To advance from here I would like to incorporate pre-trained Word Embeddings. Thus, I will extend the model to use pre-trained word2vec or GloVe vectors to see if it results in a better model. This might give the model a elementary idea of the basic syntax. 

First, I will need to find some pre-trained word vectors and download them, which is what i do below.

In [34]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt', encoding = 'utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


Next I will create a matrix containing the pre-trained word-vectors. 

In [35]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

This matrix of weights are incorporated as a layer in the model, which I train to see, if I get a different result. 

In [37]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=seq_length, trainable = False)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
#model.add(Flatten())
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           246000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 2460)              248460    
Total params: 665,360
Trainable params: 419,360
Non-trainable params: 246,000
_________________________________________________________________
None


In [38]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
model.fit(X, y, batch_size=128, epochs=1) #training the model, originally 100 epoch, but changed for illustration 

Epoch 1/1
15523/15523 [==============================] - 42s 3ms/step - loss: 6.5172 - acc: 0.0632


Next a save the model and tokenizer to prevent I will need to run the training session again. Select a seed text for the model and produce a generated output. This I save. 

In [39]:
# save the model to file
model.save('model_advanced.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [40]:
#loading model
model= load_model('model_advanced.h5')
tokenizer = load(open('tokenizer.pkl', 'rb')) #r for read

In [41]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

low ugly place but they do so no longer i dwell by the seaside says korsör i have broad highroads and pleasure gardens and i have given birth to a poet a witty one too which is more than all poets are i once thought of sending a ship all round



51

In [44]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'Data/first_output.txt'
save_doc(generated, out_filename)

the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the


This output is not from the full model, since I changed the number of iterations of the training section to allow output being shown without running the training session again. For the advanced output see appendix D. 